In [1]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 1000

In [2]:
from loadFile import getFileList, getDirList, fileToDataFrame

# ディレクトリ 内のメールファイルを読み込む
directory_path = "wiki-research-l/2020-July"
file_list = getFileList(directory_path)

# テキストファイルをデータフレームに格納する
mail_df, thread_df = fileToDataFrame(file_list)

In [3]:
from preProcess import cleaningText
s_list = []
for i,content in mail_df['Content'].items():
    sentences = cleaningText(content)
    s_list.append(sentences)
mail_df['Cleaned_Content'] = s_list

In [4]:
#　センテンスのリストを作る
sentence_list = []
for content in mail_df['Cleaned_Content']:
    sentences = tokenize.sent_tokenize(content)
    for sentence in sentences:
        sentence_list.append(sentence)

In [14]:
sent_length = {}
for i, sentence in enumerate(sentence_list):
    sent_length[i] = len(sentence.split())
print(sent_length)

{0: 9, 1: 7, 2: 3, 3: 18, 4: 12, 5: 30, 6: 24, 7: 24, 8: 25, 9: 14, 10: 70, 11: 14, 12: 19, 13: 31, 14: 2, 15: 2, 16: 13, 17: 2, 18: 25, 19: 49, 20: 10, 21: 24, 22: 11, 23: 14, 24: 15, 25: 21, 26: 40, 27: 4, 28: 4, 29: 1, 30: 27, 31: 5, 32: 18, 33: 2, 34: 30, 35: 21, 36: 34, 37: 42, 38: 25, 39: 18, 40: 10, 41: 18, 42: 10, 43: 6, 44: 21, 45: 13, 46: 17, 47: 2, 48: 23, 49: 23, 50: 7, 51: 42, 52: 14, 53: 10, 54: 52, 55: 13, 56: 23, 57: 3, 58: 7, 59: 19, 60: 3, 61: 6, 62: 11, 63: 15, 64: 5, 65: 11, 66: 16, 67: 30, 68: 37, 69: 24, 70: 4, 71: 34, 72: 110, 73: 34, 74: 24, 75: 10, 76: 54, 77: 41, 78: 136, 79: 49, 80: 1, 81: 17, 82: 13, 83: 24, 84: 53, 85: 3, 86: 3, 87: 39, 88: 13, 89: 14, 90: 21, 91: 25, 92: 15, 93: 15, 94: 15, 95: 19, 96: 1, 97: 22, 98: 6, 99: 26, 100: 13, 101: 2, 102: 5, 103: 9, 104: 51, 105: 27, 106: 87, 107: 24, 108: 58, 109: 46, 110: 36, 111: 24, 112: 51, 113: 15, 114: 24, 115: 15, 116: 18, 117: 56, 118: 36, 119: 15, 120: 27, 121: 60, 122: 5, 123: 128, 124: 15, 125: 122, 

In [22]:
sentence_list[156]

'> > > > Please fill this form to register for CICM 2020: > > > >                  https://forms.gle/oS5BVGDf6LgDGDiK8 > > > > > > SCIENTIFIC PROGRAM > > ------------------ > > > > The program of the conference is available under: > > > >              https://easychair.org/smart-program/CICM-13/ > > > > (all times are in CEST timezone (UTC+2)) > > > > > > INVITED SPEAKERS > > ------------------------ > > > > - Kevin Buzzard, Imperial College, London, UK > >   Formalizing Undergraduate Mathematics > > > > - Catherine Dubois, ENSIIE, CNRS, Evry, France > >   Formally Verified Constraints Solvers: a Guided Tour > > > > - Christian Szegedy, Google Research, Mountain View, CA, USA > >   A Promising  Path Towards  Autoformalization and  General Artificial > >   Intelligence > > > > > > INVITED WORKSHOP SPEAKERS > > ------------------------- > > > > - Freek Wiedijk, Radboud University Nijmegen, NL > >   Formal Proof for the Future > > > > - Fairouz Kamareddine, Heriot-Watt University, UK > > 

In [25]:
id_length = sorted(sent_length.items(), key=lambda x:x[1], reverse=True)
sent_cnt_list = []
for tpl in id_length:
    sent_cnt_list.append([tpl[0],tpl[1],sentence_list[tpl[0]]])

In [29]:
sent_cnt_df = pd.DataFrame(sent_cnt_list, columns=('idx','cnt','sentence'))
sent_cnt_df.to_csv('analysis/output/sent_cnt_df.csv')

In [6]:
# OpenIEにかける
from pyopenie import OpenIE5
extractor = OpenIE5('http://localhost:8000')

extractions_list = []
for sentence in sentence_list:
    try:
        extractions = extractor.extract(sentence)
        extractions_list.append(extractions)
    except Exception:
        pass

In [7]:
rows = []
for extractions in extractions_list:
    if extractions == []:
        pass
    else:
        for extraction in extractions:
            sentence = extraction['sentence']
            confidence = extraction['confidence']
            arg1 = extraction['extraction']['arg1']['text']
            rel = extraction['extraction']['rel']['text']
            arg2s_list = []
            for arg2 in extraction['extraction']['arg2s']:
                arg2s_list.append(arg2['text'])
            arg2s = ' '.join(map(str, arg2s_list))
            row = [sentence, arg1, rel, arg2s, confidence]
            rows.append(row)

In [8]:
kb_df = pd.DataFrame(rows, columns = ['sentence', 'arg1', 'rel', 'arg2s', 'confidence'])
len(kb_df)

853

In [9]:
from db import connect
engine = connect()
kb_df.to_sql(name='kb_wiki_research_l',con=engine,if_exists='replace',index=None)